In [1]:
import os, json
import openai
from tenacity import retry, stop_after_attempt, wait_random_exponential
from dotenv import load_dotenv
load_dotenv(".env")

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    QueryType,
    VectorizedQuery, 
)

### SET VARIABLES

In [2]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]

aoai_client = openai.AzureOpenAI( 
    azure_endpoint=aoai_endpoint,
    api_key=aoai_api_key,
    api_version= api_version
)
    
chat_model_ft = os.environ["AZURE_OPENAI_CHAT_MODEL_FT"]
embedding_model = os.environ["AZURE_OPENAI_EMBEDDING_MODEL"] 

service_endpoint = os.environ["SEARCH_ENDPOINT"] 
key = os.environ["SEARCH_KEY"]
search_index_name = os.environ["SEARCH_INDEX_NAME"]
credential = AzureKeyCredential(key)

search_client = SearchClient(endpoint=service_endpoint, index_name=search_index_name, credential=credential)

 ### Define Functions

In [3]:
#/*----------------*/
#  gpt4_query      */
#/*----------------*/
def gpt4_ft_query(ft_content, model=chat_model_ft):

    messages = []
    #messages.append({"role": "system","content": 'You are an AI assistant that helps people find information.'})
    messages.append({"role": "user", "content": ft_content})
    response = aoai_client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0,
    #response_format={ "type": "json_object" },
    )

    response_message = response.choices[0].message
    messages.append(response_message)

    return response_message, messages


def gpt4_query(messages, tools=None, tool_choice=None, model=chat_model_ft):

    response = aoai_client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0,
    max_tokens = 4000,
    #response_format={ "type": "json_object" },
    tools=tools,
    tool_choice=tool_choice
    )

    response_message = response.choices[0].message
    messages.append(response_message)

    return response_message, messages


def call_gpt4(messages, tools=None, tool_choice=None, user_context=None, ft_content=''):

    response_message, messages = gpt4_query(messages, tools=tools, tool_choice=tool_choice)

    answer     = response_message.content
    tool_calls = response_message.tool_calls
    #print("answer:", answer)
    print("tool_calls:", tool_calls)

    if tool_calls:
        ft_content = checkandrun_function_calling(tool_calls, messages)
        ft_content += '\n' + user_context
        response_message, messages  = gpt4_ft_query(ft_content)
    else:
        print("No tool calls were made by the model.")

    return response_message, messages, ft_content


def checkandrun_function_calling(tool_calls, messages, ft_content=''):
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        print("function_name: ", function_name)
        function_args = json.loads(tool_call.function.arguments)
        print("function_args: ", function_args)

        if function_name == "searchDocuments":
            function_response = searchDocuments(
                query=function_args.get("query")
            )
        else:
            function_response = json.dumps({"error": "Unknown function"})

        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
        ft_content += function_response
        
    return ft_content



### TOOLS DEFINITION

In [4]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text, model, aoai_client):
    text = text.replace("\n", " ")
    return aoai_client.embeddings.create(input = [text], model=model).data[0].embedding


def searchDocuments(query, aoai_client=aoai_client, embedding_model=embedding_model, search_client=search_client):

    vector_query = VectorizedQuery(vector=generate_embeddings(query, embedding_model, aoai_client), k_nearest_neighbors=3, fields="contentVector")

    results = list(search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["title", "content"],
        query_type=QueryType.SEMANTIC, 
        semantic_configuration_name="default",
        #query_caption=QueryCaptionType.EXTRACTIVE, 
        #query_answer=QueryAnswerType.EXTRACTIVE,
        top=3
    ))

    concatenated_documents = ""
    for doc in results:
        concatenated_documents += f"<DOCUMENT>\nTitle: {doc['title']}\nContent: {doc['content']}\n</DOCUMENT>\n"
    
    #concatenated_documents += '\n' + user_context

    return concatenated_documents



tools = [
    {
        "type": "function",
        "function": {
            "name": "searchDocuments",
            "description": "Use this to search for documents relevant to the query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "query string to search for documents. Use the query as it is.",
                    },
                },
                "required": ["query"],
            },
        }
    }
]

### Build message

In [5]:
with open("./100_businessSupport_sys_msg01.txt", "r", encoding = 'utf-8') as f:
    system_message = f.read()

user_context = "PCS警告灯が点滅または点灯する場合はどうすればよいですか？"

### Ask a quesion

In [6]:
messages = []
messages.append({"role": "system","content": system_message})
messages.append({"role": "user", "content": user_context})
response_message, messages, ft_content = call_gpt4(messages, tools=tools, tool_choice='auto', user_context=user_context)
print(response_message.content)

tool_calls: [ChatCompletionMessageToolCall(id='call_GM1yFaXaVdeQu7ikPLELuii6', function=Function(arguments='{"query":"PCS警告灯が点滅または点灯する場合の対処方法"}', name='searchDocuments'), type='function')]
function_name:  searchDocuments
function_args:  {'query': 'PCS警告灯が点滅または点灯する場合の対処方法'}
### 考え方

1. 質問は「PCS警告灯が点滅または点灯する場合の対処法」についてです。
2. 文脈の中で「PCS警告灯」に関する情報を探します。
3. 文脈内で「PCS警告灯」が点滅または点灯する場合の対処法が記載されている箇所を特定します。
4. 必要に応じて、関連する情報を引用し、質問に対する答えを導きます。

---

### ステップ1: 文脈内で「PCS警告灯」に関する情報を探す

文脈内で「PCS警告灯」に関する記述を見つけました。以下の内容が該当します。

##begin_quote##
PCS 警告灯 · 警告ブザーが鳴った場合: プリクラッシュセーフティの異常 →ただちにトヨタ販売店で点検を受けてください。 · 警告ブザーが鳴らない場合: プリクラッシュセーフティが一時的、または対処を行うまで作動停止している (点滅または点 灯) → マルチインフォメーションディスプレイに表示されている メッセージの指示に従ってください。(→P. 251, 567) プリクラッシュセーフティが OFF、または VSC (ビークル スタビリティコントロール)システムが停止しているときも 点灯します。 → P. 261
##end_quote##

---

### ステップ2: 情報を整理

上記の引用から、PCS警告灯が点滅または点灯する場合の対処法は以下の通りです：

1. **警告ブザーが鳴る場合**: プリクラッシュセーフティの異常が発生している可能性があるため、ただちにトヨタ販売店で点検を受ける必要があります。
2. **警告ブザーが鳴らない場合**: プリクラッシュセーフティが一時的に作動停止している可能性があ

Fllow up question

In [7]:
messages.append({"role": "user","content": "Search Index から得られたドキュメントの名前を教えてください。?"})
response_message, messages, ft_content = call_gpt4(messages, tools=tools, tool_choice='auto')
print(response_message.content)

tool_calls: None
No tool calls were made by the model.
### 考え方

1. 質問は「Search Index から得られたドキュメントの名前を教えてください」という内容です。
2. これまでの会話の中で、Search Index から取得したドキュメントが3つあります。それぞれのドキュメントにはタイトルが付けられています。
3. これらのタイトルを確認し、リストアップします。

---

### ステップ1: ドキュメントのタイトルを確認

これまでの会話で使用したドキュメントのタイトルは以下の通りです：

1. ##begin_quote##Title: prius_60##end_quote##
2. ##begin_quote##Title: prius_10##end_quote##
3. ##begin_quote##Title: prius_62##end_quote##

---

### ステップ2: 答えをまとめる

以上の情報を基に、質問に対する答えをまとめます。

---

<ANSWER>: Search Index から得られたドキュメントの名前は以下の3つです！  
1. prius_60  
2. prius_10  
3. prius_62  

これらのドキュメントを使って、質問に答えることができました！ 😊
